In [1]:
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import selenium
from time import sleep
from selenium.common.exceptions import NoSuchElementException, InvalidSelectorException
from pyvirtualdisplay import Display

vdisplay = Display(visible=1, size=(1024, 768))
vdisplay.start()

driver = webdriver.Chrome('drivers/chromedriver')
driver.get('https://instagram.com')
sleep(1.5)

username = driver.find_element(By.XPATH, '//input[@name="username"]')
username.send_keys('your username')

password = driver.find_element(By.XPATH, '//input[@name="password"]')
password.send_keys('your password')

password.send_keys(Keys.ENTER)

sleep(3)

while True:
    try:
        press = driver.find_element(By.XPATH, '//button[text() = "Not Now"]')
        press.send_keys(Keys.RETURN)
    except NoSuchElementException:
        break

/tmp/ipykernel_19187/651481952.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('drivers/chromedriver')


In [117]:
# driver.get('https://instagram.com/zuck')
# links = set()
# body = driver.find_element(By.TAG_NAME, 'body')

In [2]:
# posts[0].get_attribute('href')
driver.get('https://www.instagram.com/kairatzholdybaiuly/')
sleep(2)

# posts = driver.find_elements(By.XPATH,
#                              '//div[@style="flex-direction: column; padding-bottom: 0px; padding-top: 0px;"]/div//a[@tabindex="0"]')

posts_link = list()
prev_h = driver.execute_script('return window.pageYOffset')
body = driver.find_element(By.TAG_NAME, 'body')

while len(posts_link) < 110:
    cards = driver.find_elements(By.XPATH, '//div[@class="Nnq7C weEfm"]//a[@tabindex="0"]')

    for card in cards[-24::]:
        link = card.get_attribute('href')
        post_type = ''
        try:
            post_type = card.find_element(By.XPATH, './div[2]//*[name()="svg"]').get_attribute('aria-label')
        except NoSuchElementException:
            post_type = 'Photo '

        webdriver.ActionChains(driver).move_to_element(card).perform()

        try:
            comments = card.find_element(By.XPATH, '//div[@class="qn-0x"]/ul/li[2]/div//span').text
        except NoSuchElementException:
            comments = ''

        try:
            k = 0
            for i in posts_link:
                if link in i:
                    k += 1
            if k == 0:
                posts_link.append([link, post_type, comments])
        except:
            if link not in posts_link:
                posts_link.append([link, post_type, comments])

    body.send_keys(Keys.PAGE_DOWN)
    sleep(1)

    cur_h = driver.execute_script('return window.pageYOffset')
    if prev_h == cur_h:
        break

    prev_h = cur_h

In [3]:
def get_post_data(post, driver):
    driver.get(post[0])
    sleep(0.5)

    try:
        content = driver.find_element(By.XPATH, '//div[@class="_97aPb   wKWK0"]//video').get_attribute('src')
    except:
        content = driver.find_element(By.XPATH, '//div[@class="_97aPb   wKWK0"]//img').get_attribute('src')

    try:
        text = driver.find_element(By.XPATH, '//div[@class="EtaWk "]//div[@class="MOdxS "]').text
    except:
        text = ''

    date = driver.find_element(By.XPATH, '//div["NnvRN"]//time').get_attribute('datetime')

    if post[1].lower() == 'video':
        row = driver.find_element(By.XPATH, '//section[@class="EDfFK ygqzn "]/div//span')
        row.click()
        views = row.text
        likes = driver.find_element(By.XPATH, '//section[@class="EDfFK ygqzn "]/div/div/div[4]').text
    else:
        try:
            row = driver.find_element(By.XPATH,
                                      '//section[@class="EDfFK ygqzn "]//div[contains(text(), "others")]//span').text.replace(
                ',', '')
        except:
            row = driver.find_element(By.XPATH,
                                      '//section[@class="EDfFK ygqzn "]//div[contains(text(), "likes")]//span').text.replace(
                ',', '')

        views = ''
        likes = int(row) + 1

    content = content.replace('blob:', '')

    return [content, text, views, likes, date]



In [4]:
for card in posts_link:
    print(get_post_data(card, driver))

['https://instagram.fala7-1.fna.fbcdn.net/v/t50.16885-16/10000000_686251779104233_4362813838383277636_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6InZ0c192b2RfdXJsZ2VuLjQ4MC5pZ3R2LmJhc2VsaW5lIiwicWVfZ3JvdXBzIjoiW1wiaWdfd2ViX2RlbGl2ZXJ5X3Z0c19vdGZcIl0ifQ&_nc_ht=instagram.fala7-1.fna.fbcdn.net&_nc_cat=101&_nc_ohc=zh9GrauOo_EAX89jebR&edm=AABBvjUBAAAA&vs=1339939553179837_2315291039&_nc_vs=HBksFQAYJEdJQ1dtQURwSFlWeEpIQUNBRVJtLWlFUzFJczhidlZCQUFBRhUAAsgBABUAGCRHQXdLckJEUlVnTVpWbE1FQUlqbkZET1JRSE56YnZWQkFBQUYVAgLIAQAoABgAGwGIB3VzZV9vaWwBMRUAACaM2Jrm1t%2FXPxUCKAJDMywXQFQxBiTdLxsYEmRhc2hfYmFzZWxpbmVfMV92MREAdewHAA%3D%3D&ccb=7-4&oe=627A3716&oh=00_AT9SAMp7edSrtyvvNynGZVz_Kjzzge7QHgZfP5auyf9-1w&_nc_sid=83d603', 'Маңғыстау жұрты тасаттық беріп, Алладан сұрап, мүнәжат еткеннен кейін тарихта болмаған жаңбыр жауды. Алла ұлық!', '28,828 views', '3,990 likes', '2022-05-06T14:29:42.000Z']
['https://instagram.fala7-1.fna.fbcdn.net/v/t50.16885-16/10000000_764205784573328_8307117557650301494_n.mp4?efg=eyJ2ZW5jb2RlX3RhZyI6

In [5]:
len(posts_link)

120

In [108]:
driver.get('https://www.instagram.com/p/CWJ64wvsiXS/')
sleep(2)

row = driver.find_element(By.XPATH, '//section[@class="EDfFK ygqzn "]/div//span')
row.click()
row.text

'1,828,756 views'

In [6]:
posts_link

[['https://www.instagram.com/p/CdOE22ZlAvj/', 'Video', '179'],
 ['https://www.instagram.com/p/CdBp57xlcLb/', 'Video', '320'],
 ['https://www.instagram.com/p/Cc9z0xxN6dZ/', 'Carousel', '70'],
 ['https://www.instagram.com/p/Cc2LBsPA1pJ/', 'Video', '137'],
 ['https://www.instagram.com/p/CcxeRxFl-Ks/', 'Video', '32'],
 ['https://www.instagram.com/p/CcRVFLMh_HG/', 'Video', '71'],
 ['https://www.instagram.com/p/CcKUA8iFKt9/', 'Video', '77'],
 ['https://www.instagram.com/p/CcCWzoyFV8x/', 'Clip', '32'],
 ['https://www.instagram.com/p/CbQPj10A1LJ/', 'Video', '167'],
 ['https://www.instagram.com/p/CakZ2DVgvgb/', 'Video', '79'],
 ['https://www.instagram.com/p/CacQ4UwAEYx/', 'Photo ', '269'],
 ['https://www.instagram.com/p/CaMD-8SNpRB/', 'Photo ', '33'],
 ['https://www.instagram.com/p/CaEuykkAxFu/', 'Video', '83'],
 ['https://www.instagram.com/p/CZgp5SxhX93/', 'Video', '247'],
 ['https://www.instagram.com/p/CZgFAmgNX_m/', 'Photo ', '311'],
 ['https://www.instagram.com/p/CZFomHDr_8y/', 'Photo ', '1